In [1]:
import numpy as np

from inversion_ideas import DataMisfit, TikhonovZero, ConjugateGradient
from regressor import LinearRegressor

In [2]:
n_params = 10
rng = np.random.default_rng(seed=4242)
true_model = rng.uniform(size=10)
true_model

array([0.78225148, 0.67148671, 0.2373809 , 0.17946133, 0.34662367,
       0.15210999, 0.31142952, 0.23900652, 0.54355731, 0.91770851])

In [3]:
# Build the X array
n_data = 25
shape = (n_data, n_params)
X = rng.uniform(size=n_data * n_params).reshape(shape)

In [4]:
synthetic_data = X @ true_model
maxabs = np.max(np.abs(synthetic_data))
noise = rng.normal(scale=1e-2 * maxabs, size=synthetic_data.size)
synthetic_data += noise
synthetic_data

array([2.83840696, 2.18091081, 2.00623242, 2.08333039, 2.01694883,
       2.7826232 , 2.10564027, 1.27333506, 2.08859855, 1.94177648,
       1.88492037, 2.92394733, 2.17231952, 3.08009275, 1.61670886,
       1.77403753, 2.67305005, 1.91413882, 2.42117827, 2.13991628,
       2.0153805 , 2.71388471, 2.65944255, 2.44416121, 3.14217523])

In [5]:
simulation = LinearRegressor(X)

In [6]:
uncertainty = 1e-2 * maxabs * np.ones_like(synthetic_data)
data_misfit = DataMisfit(synthetic_data, uncertainty, simulation)
smallness = TikhonovZero(n_params)

In [7]:
phi = data_misfit + 1e-3 * smallness
phi

φ(m) + 0.00 φ(m)

In [8]:
initial_model = np.zeros(n_params)
initial_model

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

## Minimize manually with `scipy.sparse.linalg.cg`

In [10]:
from scipy.sparse.linalg import cg

In [11]:
grad = phi.gradient(initial_model)
hess = phi.hessian(initial_model)

In [12]:
inverted_model, code = cg(hess, -grad)
inverted_model, code

(array([0.80819506, 0.65382703, 0.24661944, 0.19564531, 0.32734278,
        0.1538892 , 0.32067108, 0.25559218, 0.52355838, 0.91524944]),
 0)

In [13]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)

Result:
[0.80819506 0.65382703 0.24661944 0.19564531 0.32734278 0.1538892
 0.32067108 0.25559218 0.52355838 0.91524944]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]


## Minimize with SciPy's `minimize`

In [14]:
from scipy.optimize import minimize

In [15]:
result =  minimize(phi, initial_model)
result

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 0.0029662820617797846
        x: [ 8.079e-01  6.539e-01  2.467e-01  1.955e-01  3.274e-01
             1.538e-01  3.207e-01  2.558e-01  5.234e-01  9.155e-01]
      nit: 29
      jac: [ 4.823e-06  4.106e-06  4.792e-06  9.458e-06  4.684e-06
             5.953e-06  8.263e-06  4.942e-06 -4.868e-07 -1.912e-06]
 hess_inv: [[ 6.050e+00 -2.843e+00 ...  1.314e+00 -9.830e-04]
            [-2.843e+00  9.700e+00 ... -1.975e+00  4.147e-01]
            ...
            [ 1.314e+00 -1.975e+00 ...  5.702e+00 -2.972e+00]
            [-9.830e-04  4.147e-01 ... -2.972e+00  4.399e+00]]
     nfev: 330
     njev: 30

In [16]:
print("Result:")
print(result.x)
print()
print("True model:")
print(true_model)

Result:
[0.80787092 0.65392651 0.24671057 0.19549721 0.32739929 0.15377988
 0.32069111 0.2557762  0.52343993 0.91546212]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]


In [17]:
result =  minimize(phi, initial_model, jac=phi.gradient, method="CG")
result

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 0.0029662824890714784
       x: [ 8.079e-01  6.538e-01  2.467e-01  1.954e-01  3.274e-01
            1.538e-01  3.207e-01  2.559e-01  5.235e-01  9.155e-01]
     nit: 20
     jac: [ 3.954e-06 -7.254e-06 -2.738e-07 -7.335e-06 -3.955e-07
            8.536e-07  9.889e-07  1.281e-06 -2.907e-06  1.431e-06]
    nfev: 41
    njev: 41

In [18]:
print("Result:")
print(result.x)
print()
print("True model:")
print(true_model)

Result:
[0.80790577 0.65381396 0.2466784  0.19541314 0.32739018 0.15382347
 0.32067056 0.25586811 0.52346897 0.91550894]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]


In [19]:
result =  minimize(phi, initial_model, jac=phi.gradient, hess=phi.hessian, method="Newton-CG")
result

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 0.002966281699792789
       x: [ 8.078e-01  6.539e-01  2.467e-01  1.955e-01  3.274e-01
            1.538e-01  3.207e-01  2.558e-01  5.235e-01  9.155e-01]
     nit: 6
     jac: [ 2.673e-07 -6.984e-07 -7.977e-08  4.666e-07  4.671e-07
           -3.865e-07  5.234e-07  3.062e-09 -3.999e-07 -7.682e-08]
    nfev: 7
    njev: 7
    nhev: 6

In [20]:
print("Result:")
print(result.x)
print()
print("True model:")
print(true_model)

Result:
[0.80784626 0.65391906 0.24670888 0.1954732  0.32739289 0.15377277
 0.32066917 0.25578054 0.5234611  0.9155122 ]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]


## Use `Minimizer` class

In [22]:
minimizer = ConjugateGradient()
minimizer

In [24]:
inverted_model = minimizer(phi, initial_model)
inverted_model

array([0.80819506, 0.65382703, 0.24661944, 0.19564531, 0.32734278,
       0.1538892 , 0.32067108, 0.25559218, 0.52355838, 0.91524944])

In [25]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)

Result:
[0.80819506 0.65382703 0.24661944 0.19564531 0.32734278 0.1538892
 0.32067108 0.25559218 0.52355838 0.91524944]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]
